In [1]:
import gc
import glob

import geopandas as gpd
import momepy as mm
import numpy as np
import pandas as pd
import shapely
from libpysal.graph import Graph, read_parquet
import datetime

In [2]:
regions_buildings_dir = '/data/uscuni-ulce/regions/buildings/'
buildings_dir = '/data/uscuni-ulce/processed_data/buildings/'
overture_streets_dir = '/data/uscuni-ulce/overture_streets/'
streets_dir = '/data/uscuni-ulce/processed_data/streets/'
enclosures_dir = '/data/uscuni-ulce/processed_data/enclosures/'
tessellations_dir = '/data/uscuni-ulce/processed_data/tessellations/'
graph_dir = '/data/uscuni-ulce/processed_data/neigh_graphs/'
chars_dir = '/data/uscuni-ulce/processed_data/chars/'

regions_datadir = "/data/uscuni-ulce/"
eubucco_files = glob.glob(regions_datadir + "eubucco_raw/*")

In [3]:
# prague - 69333
# freiburg - 5883
# 4 - largest region

# df = pd.read_parquet('/data/uscuni-ulce/regions/cadastre_id_to_region.parquet')
# df.region.value_counts()
# '''
# region
# 4.0         11084489
# 107131.0     3078558
# 16242.0      1457482
# 4214.0       1389195
# 8707.0       1220411
#               ...   
# 7727.0         12682
# 6741.0         12235
# 10970.0        12170
# 10847.0        10871
# 15545.0        10409
# Name: count, Length: 828, dtype: int64



# df.region.value_counts().iloc[:20].index.values.astype(int)

# array([     4, 107131,  16242,   4214,   8707,  51289, 114311,  55763,
#         14679,  62954,  10579,  38679,  86873,  34266,     10,  80404,
#         18557,  69333,   1154, 130658])
# '''

In [4]:
region_name = 69333
region_hulls = gpd.read_parquet(
        regions_datadir + "regions/" + "cadastre_regions_hull.parquet"
    )
region_id, region_hull = region_hulls.loc[region_name].name, region_hulls.loc[region_name].convex_hull

In [1]:
# region_hulls.explore()

In [5]:
region_hulls.shape

(828, 1)

## Buildings


In [28]:
from core.generate_buildings import read_region_buildings, process_region_buildings

In [41]:
%%time

buildings = gpd.read_parquet(regions_buildings_dir + f'buildings_{region_id}.pq')

CPU times: user 252 ms, sys: 77.8 ms, total: 330 ms
Wall time: 307 ms


In [31]:
%%time

buildings = process_region_buildings(buildings, True, simplification_tolerance=.1, merge_limit=25)

Percent polygons:  1.0
Final polygons:  457127 , dropped:  0.02918639964321368
CPU times: user 1min 17s, sys: 0 ns, total: 1min 17s
Wall time: 1min 17s


In [10]:

# orig_buildings = gpd.read_parquet(regions_buildings_dir + f'buildings_{region_id}.pq')
# dropped_buildings = orig_buildings[~orig_buildings['iid'].isin(buildings['iid'])]


# m = dropped_buildings.explore(color='red')
# m = buildings.explore(m=m)
# m

In [11]:
buildings.to_parquet(buildings_dir + f"buildings_{region_id}.parquet")

## Streets

In [6]:
from core.generate_streets import process_region_streets, process_single_region_streets

In [7]:
# from core.utils import largest_regions
# region_hulls = region_hulls[~region_hulls.index.isin(largest_regions)]

In [7]:
processed_region_ids = [int(s.split('.')[0].split('_')[-1])for s in glob.glob(streets_dir + '*')]
region_hulls = region_hulls[~region_hulls.index.isin(processed_region_ids)]
region_hulls

,convex_hull
labels,


In [8]:
%%time
## processs streets
streets = process_region_streets(region_id, overture_streets_dir, buildings_dir)

/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/geometry.py:247: UserWarning: Could not create a connection as it would lead outside of the artifact.
  additions, splits = snap_to_targets(
/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/geometry.py:247: UserWarning: Could not create a connection as it would lead outside of the artifact.
  additions, splits = snap_to_targets(
/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/simplify.py:343: UserWarning: An error occured at location POINT (4634734.833449237 3007874.1868347814). The artifact has not been simplified. The original message:
can only convert an array of size 1 to a Python scalar
  roads_cleaned = simplify_singletons(
/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/geometry.py:247: UserWarning: Could not create a connection as it would lead outside of the artifact.
  additions, splits = snap_to_targets(
/home/krasen/urban_taxonomy/manual_deps/sgeop/sgeop/simplify.py:200: UserWarning: Could not create a connectio

CPU times: user 9min 39s, sys: 3.21 s, total: 9min 42s
Wall time: 9min 36s


In [11]:
## save streets
streets.to_parquet(streets_dir + f'streets_{region_id}.parquet')

In [47]:
buildings.iloc[np.unique(building_idxs)].to_parquet('ws_buildings.pq')

In [48]:
streets.iloc[np.unique(idxs)].to_parquet('ws_streets.pq')

In [10]:
# streets.explore(tiles='cartodbpositron', prefer_canvas=True)

In [19]:
# import geopandas as gpd
# gpd.read_parquet(streets_dir + f'streets_{region_id}.parquet').explore()

In [20]:
# gpd.read_parquet(enclosures_dir + f'enclosure_{region_id}.parquet').explore()

## Enclosures & Tessellation

In [6]:
from core.generate_elements import process_region_elements, generate_enclosures_representative_points, generate_tess

In [7]:
# 21894
# 47090

In [8]:
# %%time
enclosures, tesselations = process_region_elements(buildings_dir, streets_dir, region_id)

---- Processing region:  21894 2024-10-12 19:08:34.123948



KeyboardInterrupt



In [29]:
region_hulls = gpd.read_parquet(
    regions_datadir + "regions/" + "cadastre_regions_hull.parquet"
)
processed_region_ids = [int(s.split('.')[0].split('_')[-1])for s in glob.glob(tessellations_dir + '*')]
region_hulls = region_hulls[~region_hulls.index.isin(processed_region_ids)]
region_hulls

,convex_hull
labels,


In [13]:
def process_single_region_elements(region_id):
    enclosures, tesselations = process_region_elements(buildings_dir, streets_dir, region_id)
    
    enclosures.to_parquet(enclosures_dir + f"enclosure_{region_id}.parquet")
    print("Processed enclosures")
    
    ## save files
    tesselations.to_parquet(
        tessellations_dir + f"tessellation_{region_id}.parquet"
    )
    print("processed tesselations")

In [14]:
# for region_id, _ in region_hulls.iterrows():
#     process_single_region_elements(region_id)

In [ ]:
%%capture cap

from joblib import Parallel, delayed
n_jobs = -1
new = Parallel(n_jobs=n_jobs)(
    delayed(process_single_region_elements)(region_id) for region_id, _ in region_hulls.iterrows()
)

/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/libpysal/graph/_triangulation.py:265: DeprecationWarning: `row_stack` alias is deprecated. Use `np.vstack` directly.
  edges = numpy.row_stack(list(set(map(tuple, edges)).difference(set(droplist))))
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/libpysal/graph/_triangulation.py:265: DeprecationWarning: `row_stack` alias is deprecated. Use `np.vstack` directly.
  edges = numpy.row_stack(list(set(map(tuple, edges)).difference(set(droplist))))
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/libpysal/graph/_triangulation.py:265: DeprecationWarning: `row_stack` alias is deprecated. Use `np.vstack` directly.
  edges = numpy.row_stack(list(set(map(tuple, edges)).difference(set(droplist))))
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/libpysal/graph/_triangulation.py:265: DeprecationWarning: `row_stack` alias is deprecated. Use `np.

---- Processing region:  53525 2024-10-12 19:25:18.890283
Processed enclosures
processed tesselations
---- Processing region:  55811 2024-10-12 19:27:22.948510
Processed enclosures
processed tesselations
---- Processing region:  57313 2024-10-12 19:30:22.096216
Processed enclosures
processed tesselations


/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/libpysal/graph/_triangulation.py:265: DeprecationWarning: `row_stack` alias is deprecated. Use `np.vstack` directly.
  edges = numpy.row_stack(list(set(map(tuple, edges)).difference(set(droplist))))
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/libpysal/graph/_triangulation.py:265: DeprecationWarning: `row_stack` alias is deprecated. Use `np.vstack` directly.
  edges = numpy.row_stack(list(set(map(tuple, edges)).difference(set(droplist))))
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/libpysal/graph/_triangulation.py:265: DeprecationWarning: `row_stack` alias is deprecated. Use `np.vstack` directly.
  edges = numpy.row_stack(list(set(map(tuple, edges)).difference(set(droplist))))


---- Processing region:  53744 2024-10-12 19:25:18.890076
Processed enclosures
processed tesselations
---- Processing region:  56178 2024-10-12 19:28:05.479164
Processed enclosures
processed tesselations
---- Processing region:  56937 2024-10-12 19:30:06.324060
Processed enclosures
processed tesselations


/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/libpysal/graph/_triangulation.py:265: DeprecationWarning: `row_stack` alias is deprecated. Use `np.vstack` directly.
  edges = numpy.row_stack(list(set(map(tuple, edges)).difference(set(droplist))))
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/libpysal/graph/_triangulation.py:265: DeprecationWarning: `row_stack` alias is deprecated. Use `np.vstack` directly.
  edges = numpy.row_stack(list(set(map(tuple, edges)).difference(set(droplist))))
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/libpysal/graph/_triangulation.py:265: DeprecationWarning: `row_stack` alias is deprecated. Use `np.vstack` directly.
  edges = numpy.row_stack(list(set(map(tuple, edges)).difference(set(droplist))))
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/libpysal/graph/_triangulation.py:265: DeprecationWarning: `row_stack` alias is deprecated. Use `np.

---- Processing region:  52492 2024-10-12 19:25:18.761455
Retrying tesselation with less buildings, potentially changing building data.
Dropping 1 buildings due to tesselation problems
Processed enclosures
processed tesselations
---- Processing region:  56184 2024-10-12 19:28:14.916295
Retrying tesselation with less buildings, potentially changing building data.
Dropping 14 buildings due to tesselation problems
Processed enclosures
processed tesselations


/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/libpysal/graph/_triangulation.py:265: DeprecationWarning: `row_stack` alias is deprecated. Use `np.vstack` directly.
  edges = numpy.row_stack(list(set(map(tuple, edges)).difference(set(droplist))))


---- Processing region:  51677 2024-10-12 19:25:18.761454
Retrying tesselation with less buildings, potentially changing building data.
Dropping 1 buildings due to tesselation problems
Processed enclosures
processed tesselations
---- Processing region:  55835 2024-10-12 19:27:51.912350
Retrying tesselation with less buildings, potentially changing building data.
Dropping 2 buildings due to tesselation problems
Processed enclosures
processed tesselations


In [16]:
1

1

In [18]:
1

---- Processing region:  99865 2024-10-12 20:19:50.054344
Retrying tesselation with less buildings, potentially changing building data.
Dropping 2 buildings due to tesselation problems
Processed enclosures
processed tesselations
---- Processing region:  105589 2024-10-12 20:34:22.636470
Processed enclosures
processed tesselations
---- Processing region:  107131 2024-10-12 20:36:29.793995
Retrying tesselation with less buildings, potentially changing building data.
Dropping 1 buildings due to tesselation problems
Processed enclosures
processed tesselations


1

In [12]:
# new_buildings = gpd.read_parquet(buildings_dir + f'buildings_{region_id}.parquet')

# # m = buildings.explore(max_zoom=20)
# # m = new_buildings.explore(m=m, color='red')


# # m = buildings[~buildings['iid'].isin(new_buildings['iid'])].explore(m=m, color='red')

# # m = dropped_buildings.explore(m=m, color='red')
# # m = enclosures.explore(m=m, color='green')
# # m = tesselations[tesselations['enclosure_index'] == 211].reset_index().explore(m=m, color='green')
# # m

In [13]:
enclosures.to_parquet(enclosures_dir + f"enclosure_{region_id}.parquet")
print("Processed enclosures")

## save files
tesselations.to_parquet(
    tessellations_dir + f"tessellation_{region_id}.parquet"
)
print("processed tesselations")

Processed enclosures
processed tesselations


In [134]:
# import lonboard
# layer = lonboard.PolygonLayer.from_geopandas(tesselations, opacity=0.15)
# m = lonboard.Map([layer])
# m

In [22]:
# layer = lonboard.PolygonLayer.from_geopandas(enclosures, opacity=0.15)
# m = lonboard.Map([layer])
# m

## Graphs

In [19]:
from core.generate_ngraphs import process_region_graphs

In [12]:
%%time
process_region_graphs(
    region_id,
    graph_dir,
    buildings_dir,
    streets_dir,
    enclosures_dir,
    tessellations_dir,
)

Built tess graph knn=1
Built buildings graph knn=1
Built streets graph knn=1
Built enclosure graph knn=1
Built nodes graph knn=1
CPU times: user 1.28 s, sys: 27.9 ms, total: 1.31 s
Wall time: 1.3 s


/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/libpysal/weights/weights.py:1685: UserWarning: The weights matrix is not fully connected: 
 There are 19 disconnected components.
  w = W(neighbors, weights, ids, silence_warnings=silence_warnings)


In [ ]:
processed_region_ids = [int(s.split('.')[0].split('_')[-1])for s in glob.glob(graph_dir + '*')]

In [35]:
region_hulls = gpd.read_parquet(
    regions_datadir + "regions/" + "cadastre_regions_hull.parquet"
)
processed_region_ids = [int(s.split('.')[0].split('_')[-1])for s in glob.glob(graph_dir + '*')]
region_hulls = region_hulls[~region_hulls.index.isin(processed_region_ids)]
region_hulls

,convex_hull
labels,
21894,"POLYGON ((4290400 3444300, 4290000 3444400, 42..."
21904,"POLYGON ((4298300 3013900, 4293400 3014600, 42..."
22040,"POLYGON ((4300900 3137100, 4295800 3137600, 42..."
22105,"POLYGON ((4304900 2870800, 4298400 2872300, 42..."
22345,"POLYGON ((4308200 3070700, 4298500 3071500, 42..."
...,...
50528,"POLYGON ((4492200 3191700, 4486500 3193600, 44..."
50880,"POLYGON ((4494600 3230000, 4492100 3230500, 44..."
50896,"POLYGON ((4506400 3167400, 4503300 3168700, 44..."


In [ ]:
from joblib import Parallel, delayed
n_jobs = -1
new = Parallel(n_jobs=n_jobs)(
    delayed(process_region_graphs)(     region_id,
                                        graph_dir,
                                        buildings_dir,
                                        streets_dir,
                                        enclosures_dir,
                                        tessellations_dir
                                  ) 
    for region_id, _ in region_hulls.iterrows())

In [38]:
1

1

## Characters

In [13]:
from core.generate_chars import process_single_region_chars, process_street_chars

In [14]:
%%time
process_single_region_chars(
    region_id,
    graph_dir,
    buildings_dir,
    streets_dir,
    enclosures_dir,
    tessellations_dir,
    chars_dir
)

2024-10-12 10:03:09.505871 ----Processing ------ 7727
Processing streets
Processing enclosures
Processing buildings


/home/krasen/urban_taxonomy/manual_deps/momepy/momepy/functional/_shape.py:576: RuntimeWarning: Mean of empty slice
  return np.nanmean(np.abs(90 - degrees[true_angles]))
/home/krasen/urban_taxonomy/manual_deps/momepy/momepy/functional/_shape.py:737: RuntimeWarning: Mean of empty slice
  return Series({"mean": np.nanmean(dists), "std": np.nanstd(dists)})
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Processing tessellation
CPU times: user 52.4 s, sys: 149 ms, total: 52.5 s
Wall time: 52.5 s


## Generate primary data

In [6]:
from core.generate_merged_primary_chars import merge_into_primary

In [7]:
%%time
merge_into_primary(region_id)

CPU times: user 172 ms, sys: 54 ms, total: 225 ms
Wall time: 204 ms


In [29]:
tessellation = gpd.read_parquet(chars_dir + f"tessellations_chars_{region_id}.parquet")
buildings = gpd.read_parquet(chars_dir + f"buildings_chars_{region_id}.parquet")
enclosures = gpd.read_parquet(chars_dir + f"enclosures_chars_{region_id}.parquet")
streets = gpd.read_parquet(chars_dir + f"streets_chars_{region_id}.parquet")
nodes = gpd.read_parquet(chars_dir + f"nodes_chars_{region_id}.parquet")

In [30]:

merged = pd.merge(
    tessellation.drop(columns=["geometry"]),
    buildings.drop(columns=["nodeID", "geometry", 'nID']),
    right_index=True,
    left_index=True,
    how="left",
)

merged = merged.merge(
    enclosures.drop(columns="geometry"),
    right_on="eID",
    left_on="enclosure_index",
    how="left",
)

merged = merged.merge(streets.drop(columns="geometry"), on="nID", how="left")
merged = merged.merge(nodes.drop(columns="geometry"), on="nodeID", how="left")

merged = merged.drop(
    columns=[
        "nID",
        "eID",
        "nodeID",
        "mm_len",
        "cdsbool",
        "node_start",
        "node_end",
        "x",
        "y",
        "enclosure_index",
        # "id",
        # "osm_id",
        "index",  ## maybe keep
    ]
)
merged = merged.set_index(tessellation.index)

In [31]:
from core.utils import used_keys

In [32]:
primary = merged[list(used_keys.keys())]
primary.shape

(459980, 63)

In [33]:
primary.to_parquet(chars_dir + f'primary_chars_{region_id}.parquet')

## Generate morphotopes

In [6]:
from core.generate_clusters import process_single_region_morphotopes

In [7]:
process_single_region_morphotopes(region_id)

2024-10-13 19:15:14.977281 ----Processing ------ 69333
--------Generating lag----------
--------Generating morphotopes----------
--------Generating morphotopes data----------
